# Import Libraries

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plot

# Dataset

### Dataset Location

In [ ]:
TrainFolder = r'C:\Users\Aakash Maurya\Documents\Leaf Disease Detection\Dataset\train'
TestFolder = r'C:\Users\Aakash Maurya\Documents\Leaf Disease Detection\Dataset\val'

In [ ]:
IMG_SIZE = (112, 112)
BATCH_SIZE = 32

TrainData = tf.keras.preprocessing.image_dataset_from_directory(
    directory = TrainFolder,
    image_size = IMG_SIZE,
    label_mode = 'categorical',
    batch_size = BATCH_SIZE,
#     shuffle = True,
#     seed=42
).cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

TestData = tf.keras.preprocessing.image_dataset_from_directory(
    directory = TestFolder,
    image_size = IMG_SIZE,
    label_mode = 'categorical',
    batch_size = BATCH_SIZE
)

ClassList = TestData.class_names
ValidData = TestData.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
plot.figure(figsize=(18,10))
for ImageBatch, LabelBatch in TrainData.take(1):
    for i in range(0,12):
        plot.subplot(3,4,i+1)
        plot.imshow(ImageBatch[i].numpy().astype("uint32"))
        plot.title(ClassList[np.argmax(LabelBatch[i])])
        plot.axis("off")

### Model for Data Augmentation

In [ ]:
DataAugmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(112, 112, 3)),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
  tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
  tf.keras.layers.experimental.preprocessing.RandomHeight(0.2),
  tf.keras.layers.experimental.preprocessing.RandomWidth(0.2),
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
], name ="DataAugmentation")

### Result of Data Augmentation

In [ ]:
plot.figure(figsize=(10,10))
for ImageBatch, LabelBatch in TrainData.take(1):
    AugmentedImage = DataAugmentation(ImageBatch)
    ImagePlotLen = len(ImageBatch) if len(ImageBatch) < 4 else 4
    for i in range(0, ImagePlotLen):
        plot.subplot(ImagePlotLen, 2, 2*i+1)
        plot.imshow(ImageBatch[i].numpy().astype("uint32"))
        plot.title(ClassList[np.argmax(LabelBatch[i])])
        plot.axis("off")
        plot.subplot(ImagePlotLen, 2, 2*i+2)
        plot.imshow(AugmentedImage[i])
        plot.title(f"augmented")
        plot.axis("off")

In [ ]:
Checkpoint = "CheckPoint/cp.ckpt"
Checkpoint_Callback = tf.keras.callbacks.ModelCheckpoint(
    Checkpoint,
    save_weights_only=True,
    monitor='val_accuracy',
    save_best_only=True
)

# Model Creation

### Custom

In [ ]:
Model = tf.keras.Sequential([
    DataAugmentation,
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2),
    tf.keras.layers.Conv2D(16, 3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(len(ClassList), activation='softmax', name='output_layers')
])

In [ ]:
Model.compile(
    loss = 'categorical_crossentropy',
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy']
)

In [ ]:
History = Model.fit(
    TrainData,
    epochs=100,
    steps_per_epoch=len(TrainData),
    validation_data = TestData,
    validation_steps = len(TestData),
    callbacks = [
        Checkpoint_Callback
    ]
)

In [ ]:
Model.summary()

### Pretrained

In [ ]:
Model.load_weights(Checkpoint)

In [ ]:
Model.evaluate(TrainData), Model.evaluate(TestData)

In [ ]:
Pretrained_Model = tf.keras.models.load_model("Pretrained Model.h5")

In [ ]:
Pretrained_Model.summary()

# Model Comparison

In [ ]:
Model.evaluate(TrainData), Model.evaluate(TestData)

In [ ]:
Pretrained_Model.evaluate(TrainData), Pretrained_Model.evaluate(TestData)